In [1]:
import os
import numpy as np
import time
import json
import boto3
import requests
import scipy.io
import zipfile
#import schedule
import matlab.engine
from requests.auth import HTTPBasicAuth
from sqlalchemy import create_engine, select, MetaData, Table, and_, update
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy.sql.expression import func


wj_engine = create_engine("********")
metadata = MetaData(bind=None)
table = Table('********',  metadata,  autoload=True,  autoload_with=wj_engine)
stmt = select([table.columns.'********'])
 
    

In [2]:

def getinfo_updateem(newdata, TARGETINT):
    parent =newdata[TARGETINT][0] 
    name = newdata[TARGETINT][1]
    uuid = newdata[TARGETINT][2]
    result = newdata[TARGETINT][3]
    snr = newdata[TARGETINT][4]
    created = newdata[TARGETINT][5]
    updaetd = newdata[TARGETINT][6]
    analysis = newdata[TARGETINT][8]
    gain = newdata[TARGETINT][10]

    wj_engine = create_engine("********")
    metadata = MetaData(bind=None)
    table = Table('********',  metadata,  autoload=True,  autoload_with=wj_engine)
    stmt = select([table]).where(and_(table.c.parentUUID_id == parent))
    wjpersoninfo = wj_engine.connect().execute(stmt).fetchone()

    sch_engine = create_engine("********")
    metadata = MetaData(bind=None)
    table =Table('person', metadata, autoload=True,  autoload_with=sch_engine)
    import datetime
    today = datetime.datetime.today()
    stmt = table.insert().values(name = wjpersoninfo[2],
                                                            birth = wjpersoninfo[4],
                                                            gender = wjpersoninfo[3],
                                                            grade =wjpersoninfo[-1],
                                                            measure_place = 'Fiji',
                                                            registered = today,
                                                            uuid = uuid,
                                                            test = True,
                                                             total_file_uploaded=True, report_completed=False)
    connection = sch_engine.connect()
    connection.execute(stmt)

    table =Table('person', metadata, autoload=True,  autoload_with=sch_engine)
    stmt = select([table.c.id]).where(table.c.name == wjpersoninfo[2])
    connection = sch_engine.connect()
    person_id = connection.execute(stmt).fetchall()[0][0]

    import datetime
    table = Table('task', metadata, autoload =True, autoload_with = sch_engine)
    stmt = table.insert().values(uuid = uuid,
                                                            result = '100',
                                                            snr = snr,
                                                            gain = gain,
                                                            analysis = None,
                                                            created = created,
                                                            updated =updaetd,
                                                            app_id = 17,
                                                            measurement_id = '122',
                                                            person_id = person_id
                                                            )
    connection = sch_engine.connect()
    connection.execute(stmt)


In [5]:
# main
# get UUID list from woongjin and download files. if files are all there transfer student info to seocho db


table = Table('********',  metadata,  autoload=True,  autoload_with=wj_engine)
# stmt = select([table.c.taskUUID]).where(and_(table.c.taskUUID.startswith('1'), 
#                                              func.length(table.c.taskUUID) == 8,
#                                              func.length(table.c.childName) ==3,
#                                             table.c.result == '100'))
stmt = select([table]).where(and_(table.c.taskUUID.startswith('1'), 
                                             func.length(table.c.taskUUID) == 8,
                                             func.length(table.c.childName) ==3,
                                            table.c.result == '100'))
connection = wj_engine.connect()
newdata = connection.execute(stmt).fetchall()

import glob
import json
import boto3
key = "********"
with open(key) as json_file:
    data = json.load(json_file)
ID = data['AWS_ACCESS_KEY_ID']
KEY = data['AWS_SECRET_ACCESS_KEY']

for i in range(16,36):
    TARGETINT = i
    uuid = newdata[TARGETINT][2]
    print(uuid)
    try: 
        get_data_url = '********'
        r = requests.get(get_data_url, allow_redirects = True, auth=HTTPBasicAuth('admin', 'obe1234'))
        open(f"{uuid}.zip", 'wb').write(r.content) # download
        with zipfile.ZipFile(f"{uuid}.zip", "r") as zip_ref: # unzip
            zip_ref.extractall(f"{uuid}")

        output_path = os.getcwd()+f'\\{uuid}'
        s3 = boto3.client('s3',aws_access_key_id=ID,aws_secret_access_key= KEY)
        csvfiles = glob.glob(output_path + '\\*.csv')
        if len(csvfiles) > 8:
            print(f'processing {uuid}')
            for file in csvfiles:
                if file.split('_')[1] in ['01','02','07','08']: # upload only the target tasks
                    filename = ('/').join(file.split('\\')[3:])
                    print(filename)                            
                    s3.upload_file( file, '********', 'csv/' + filename)
            a =getinfo_updateem(newdata, TARGETINT)
        else:
            print(f'filelength incorrect, skipping {uuid}')
            print(len(csvfiles))
    except:
        print(f'error, skipping {uuid}')

1cYjeFAF
processing 1cYjeFAF
1cYjeFAF/1cYjeFAF_01_01_marker.csv
1cYjeFAF/1cYjeFAF_01_01_raw.csv
1cYjeFAF/1cYjeFAF_02_01_marker.csv
1cYjeFAF/1cYjeFAF_02_01_raw.csv
1cYjeFAF/1cYjeFAF_07_01_marker.csv
1cYjeFAF/1cYjeFAF_07_01_raw.csv
1cYjeFAF/1cYjeFAF_08_01_marker.csv
1cYjeFAF/1cYjeFAF_08_01_raw.csv
error, skipping 1cYjeFAF
1byRYQBA
processing 1byRYQBA
1byRYQBA/1byRYQBA_01_01_marker.csv
1byRYQBA/1byRYQBA_01_01_raw.csv
1byRYQBA/1byRYQBA_02_01_marker.csv
1byRYQBA/1byRYQBA_02_01_raw.csv
1byRYQBA/1byRYQBA_07_01_marker.csv
1byRYQBA/1byRYQBA_07_01_raw.csv
1byRYQBA/1byRYQBA_08_01_marker.csv
1byRYQBA/1byRYQBA_08_01_raw.csv
error, skipping 1byRYQBA
1BwavXWU
processing 1BwavXWU
1BwavXWU/1BwavXWU_01_01_marker.csv
1BwavXWU/1BwavXWU_01_01_raw.csv
1BwavXWU/1BwavXWU_02_01_marker.csv
1BwavXWU/1BwavXWU_02_01_raw.csv
1BwavXWU/1BwavXWU_07_01_marker.csv
1BwavXWU/1BwavXWU_07_01_raw.csv
1BwavXWU/1BwavXWU_08_01_marker.csv
1BwavXWU/1BwavXWU_08_01_raw.csv
error, skipping 1BwavXWU
1CHH28TW
processing 1CHH28TW
1CHH2

In [6]:
import scipy.io
uuids = [i[2] for i in newdata]
uuids  = { 'uuids' : np.array(uuids, dtype=object) }
scipy.io.savemat('neIDs.mat', uuids)

'E:\\seochoReport\\notebook\\1ADFGvT3'